<a href="https://colab.research.google.com/github/lpoggetto/trabalho_nlp_fiap/blob/main/trabalho_nlp_fiap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

***Participantes (RM - NOME):***<br>
Cleber Veiga Galvão - 356846<br>
Danildo André Amorim dos Santos - 355182<br>
Victor Marcio Hayee Falcão - 354955<br>
Lucas Ribeiro Del Poggetto - 355220<br>

###**Crie um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para alguma área especialista no assunto para uma melhor tratativa.​

1. Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do seu modelo.​

  O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

2. Utilizar ao menos uma aplicação de modelos de GenAI (LLM´s) para criar o modelo classificador com os mesmos critérios do item 1.

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, aplicações de GenIA, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais serão descontado nota e passível de reprovação.

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

Bom desenvolvimento!

###**Area de desenvolvimento e validações**

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

In [1]:
# limpando a pasta
!rm -rf /content/*

# Pacotes utilizados em NLP/feature engineering
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import re

import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
!python -m spacy download pt_core_news_sm

# pacote ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# genai

import torch
import torchvision  # Import torchvision before transformers
# Workaround for the circular import issue
if hasattr(torchvision, 'disable_beta_transforms_warning'):
    torchvision.disable_beta_transforms_warning()
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


# importando os dados do trabalho
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


In [3]:
df.shape

(21072, 4)

In [4]:
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


In [5]:
df['categoria'].value_counts()

,count
categoria,
Serviços de conta bancária,5161
Cartão de crédito / Cartão pré-pago,5006
Roubo / Relatório de disputa,4822
Hipotecas / Empréstimos,3850
Outros,2233


In [6]:
df.isnull().sum()

,0
id_reclamacao,0
data_abertura,0
categoria,0
descricao_reclamacao,0


### Criando funções para limpeza/normalização dos dados


In [25]:
# inicializar o spacy
nlp = spacy.load("pt_core_news_sm")

# padroes regex manipulacao dos dados
URL_PATTERN = re.compile(r'http\S+|www\S+|https\S+', flags=re.MULTILINE)
SPECIAL_CHARS_PATTERN = re.compile(r'[^\w\s]')
NUMBERS_PATTERN = re.compile(r'\d+')
DATES_PATTERN = re.compile(r'\d{2}/\d{2}/\d{4}')
WHITESPACE_PATTERN = re.compile(r'\s+')

# operacoes regex
def combined_regex(texto):
    texto = SPECIAL_CHARS_PATTERN.sub('', texto)
    texto = NUMBERS_PATTERN.sub('[NUMERO]', texto)
    texto = URL_PATTERN.sub('', texto)
    texto = DATES_PATTERN.sub('[DATA]', texto)
    texto = WHITESPACE_PATTERN.sub(' ', texto).strip()
    texto = texto.lower()
    return texto

# convertendo stopwords para um frozenset - maior velocidade
STOPWORDS_PT = frozenset(STOP_WORDS)

# remocao de stopwords
def remove_stopwords(texto):
    return " ".join(word for word in texto.split() if word not in STOPWORDS_PT)

def lemmatize_text(df, text_column='texto', output_column='text_lemma'):
    # lista vazia para textos lemmatizados
    lemma_text_list = []

    # desabilitando funcoes nao utilizadas
    with nlp.select_pipes(disable=["parser", "ner"]):
        # processando em batches para aumentar a eficiencia
        for doc in nlp.pipe(df[text_column].astype(str).fillna(''), batch_size=50):
            # agregando os lemmas
            lemma_text_list.append(" ".join(token.lemma_ for token in doc))

    # criando uma copia do df
    df = df.copy()
    # resultados aplicados em uma nova coluna
    df[output_column] = lemma_text_list

    return df

# preprocessamento
def preprocessamento(texto):
    texto = combined_regex(texto)
    texto = remove_stopwords(texto)
    return texto

In [8]:
# aplicando o preprocessamento de texto
df['descricao_reclamacao_teste'] = df['descricao_reclamacao'].apply(preprocessamento)

# lemmatizando texto
df = lemmatize_text(df, text_column='descricao_reclamacao_teste', output_column='descricao_reclamacao_teste_lemma')

In [10]:
# 2. Codificar as categorias (variável resposta)
le = LabelEncoder()
df['categoria_encoded'] = le.fit_transform(df['categoria'])

# 3. Dividir os dados
X = df['descricao_reclamacao_teste_lemma']  # textos pré-processados
y = df['categoria_encoded']  # categorias codificadas numericamente

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=y
                                                    )

# 4. Criar pipeline de classificação
model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # vetorização TF-IDF
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))  # classificador
])

# 5. Treinar o modelo
model.fit(X_train, y_train)

# 6. Avaliar o modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.86      0.89      0.88      1252
            Hipotecas / Empréstimos       0.88      0.89      0.88       962
                             Outros       0.91      0.61      0.73       558
       Roubo / Relatório de disputa       0.85      0.84      0.85      1206
         Serviços de conta bancária       0.83      0.92      0.87      1290

                           accuracy                           0.86      5268
                          macro avg       0.87      0.83      0.84      5268
                       weighted avg       0.86      0.86      0.85      5268



In [11]:
# 7. Exemplo de previsão para novo texto
novo_texto = "Fui hackeado e levaram 1000 reais e gostaria de reaver o valor perdido visto que não a transação não foi autorizada"
novo_texto_processado = preprocessamento(novo_texto)
categoria_predita = model.predict([novo_texto_processado])
print(f"Categoria prevista: {le.inverse_transform(categoria_predita)[0]}")

Categoria prevista: Roubo / Relatório de disputa


#### Aplicando GenAI

In [12]:
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [13]:
# Tokenizando os textos
# Esse processo é para facilitar o processamento com o modelo BERT a seguir, confertendo os textos em tensores (vetores muiti camadas)

# Inicializando o tokenizador BERT para o português
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [14]:
# Seperação dos dados de treino e teste
## Dados de treino
train_texts = train_df['descricao_reclamacao_teste_lemma'].tolist()
train_labels = train_df['categoria'].tolist()

## Dados de teste
test_texts = test_df['descricao_reclamacao_teste_lemma'].tolist()
test_labels = test_df['categoria'].tolist()

In [15]:
# Codificando os textos usando o tokenizador
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)

In [17]:
# Codificando as categorias em números e convertendo as categorias para IDs numéricos
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

In [18]:
# Convertendo os dados para tensores PyTorch
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

In [19]:
# Inicializando o modelo BERT pré-treinado para classificação
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(set(train_labels)))


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Inicializando o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

In [21]:
# Inicializando o DataLoader para treinamento e teste
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [22]:
# Treinamento do modelo
## identifica qual o processador da sessão (CPU ou GPU?), recomendo usar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 3

for epoch in range(num_epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Os IDs correspondem a índices no vocabulário do modelo BERT
#Internamente, o modelo BERT tem uma camada de embeddings (chamada Embedding Layer), que converte esses IDs em vetores densos de alta dimensão.



Epoch 1:   0%|          | 3/3951 [00:23<8:46:20,  8.00s/it]


KeyboardInterrupt: 

In [27]:
# Avaliação do modelo
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Mapeando os IDs preditos de volta para as categorias originais usando inverse_transform
#predicted_labels = all_predictions
#true_labels = all_labels
predicted_labels = le.inverse_transform(all_predictions)
true_labels = le.inverse_transform(all_labels)
#len(test_dataset)
#len(predicted_labels)



Evaluating: 100%|██████████| 1317/1317 [00:39<00:00, 33.32it/s]

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.85      0.87      0.86      1290
            Hipotecas / Empréstimos       0.85      0.89      0.87       922
                             Outros       0.76      0.79      0.77       549
       Roubo / Relatório de disputa       0.87      0.78      0.82      1204
         Serviços de conta bancária       0.86      0.88      0.87      1303

                           accuracy                           0.85      5268
                          macro avg       0.84      0.84      0.84      5268
                       weighted avg       0.85      0.85      0.85      5268



In [28]:
# Exibindo as métricas de classificação
print(classification_report(true_labels, predicted_labels))

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.85      0.87      0.86      1290
            Hipotecas / Empréstimos       0.85      0.89      0.87       922
                             Outros       0.76      0.79      0.77       549
       Roubo / Relatório de disputa       0.87      0.78      0.82      1204
         Serviços de conta bancária       0.86      0.88      0.87      1303

                           accuracy                           0.85      5268
                          macro avg       0.84      0.84      0.84      5268
                       weighted avg       0.85      0.85      0.85      5268



###**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

**Atenção:** **Implemente aqui apenas os scripts que fazem parte do modelo campeão.**


In [2]:
# instalando pacote para stopwords em portugues do spacy
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 82.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Pacotes utilizados em NLP/feature engineering
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
import re

import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

# pacote ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# GenAI e DeepLearning
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# importando os dados do trabalho
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

# criacao de funcoes para limpeza, normalizacao e lemmatizacao dos textos
# inicializar o spacy
nlp = spacy.load("pt_core_news_sm")

# padroes regex manipulacao dos dados
URL_PATTERN = re.compile(r'http\S+|www\S+|https\S+', flags=re.MULTILINE)
SPECIAL_CHARS_PATTERN = re.compile(r'[^\w\s]')
NUMBERS_PATTERN = re.compile(r'\d+')
DATES_PATTERN = re.compile(r'\d{2}/\d{2}/\d{4}')
WHITESPACE_PATTERN = re.compile(r'\s+')

# operacoes regex
def combined_regex(texto):
    texto = SPECIAL_CHARS_PATTERN.sub('', texto)
    texto = NUMBERS_PATTERN.sub('[NUMERO]', texto)
    texto = URL_PATTERN.sub('', texto)
    texto = DATES_PATTERN.sub('[DATA]', texto)
    texto = WHITESPACE_PATTERN.sub(' ', texto).strip()
    texto = texto.lower()
    return texto

# convertendo stopwords para um frozenset - maior velocidade
STOPWORDS_PT = frozenset(STOP_WORDS)

# remocao de stopwords
def remove_stopwords(texto):
    return " ".join(word for word in texto.split() if word not in STOPWORDS_PT)

def lemmatize_text(df, text_column='texto', output_column='text_lemma'):
    # lista vazia para textos lemmatizados
    lemma_text_list = []

    # desabilitando funcoes nao utilizadas
    with nlp.select_pipes(disable=["parser", "ner"]):
        # processando em batches para aumentar a eficiencia
        for doc in nlp.pipe(df[text_column].astype(str).fillna(''), batch_size=50):
            # agregando os lemmas
            lemma_text_list.append(" ".join(token.lemma_ for token in doc))

    # criando uma copia do df
    df = df.copy()
    # resultados aplicados em uma nova coluna
    df[output_column] = lemma_text_list

    return df

# preprocessamento do texto
def preprocessamento(texto):
    texto = combined_regex(texto)
    texto = remove_stopwords(texto)
    return texto

# aplicando o preprocessamento de texto - descricao_reclamacao_teste_lemma
df['descricao_reclamacao_teste'] = df['descricao_reclamacao'].apply(preprocessamento)

# lemmatizando texto
df = lemmatize_text(df, text_column='descricao_reclamacao_teste', output_column='descricao_reclamacao_teste_lemma')

# 4. Separando dados de treino e teste
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

#5. Tokenizando os textos
# Inicializando o tokenizador BERT para o português
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

#6. Seperação dos dados de treino e teste
## Dados de treino
train_texts = train_df['descricao_reclamacao_teste_lemma'].tolist()
train_labels = train_df['categoria'].tolist()
## Dados de teste
test_texts = test_df['descricao_reclamacao_teste_lemma'].tolist()
test_labels = test_df['categoria'].tolist()

#7.1. Codificando os textos usando o tokenizador
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=200)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=200)

#7.2. Codificando as categorias em números e convertendo as categorias para IDs numéricos
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

#8. Convertendo os dados para tensores PyTorch
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

#9.1. Inicializando o modelo BERT pré-treinado para classificação
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(set(train_labels)))

#9.2. Inicializando o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

#9.3. Inicializando o DataLoader para treinamento e teste
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#10. Treinamento do modelo
## identifica qual o processador da sessão (CPU ou GPU?)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 3

for epoch in range(num_epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Epoch 1:   0%|          | 0/317 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Epoch 3: 100%|██████████| 317/317 [01:48<00:00,  2.93it/s]


In [5]:
#11. Avaliação do modelo
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Mapeando os IDs preditos de volta para as categorias originais usando inverse_transform
predicted_labels = le.inverse_transform(all_predictions)
true_labels = le.inverse_transform(all_labels)


Evaluating: 100%|██████████| 106/106 [00:11<00:00,  9.32it/s]


In [6]:
# Exibindo as métricas de classificação
print(classification_report(true_labels, predicted_labels))

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.91      0.86      0.88      1290
            Hipotecas / Empréstimos       0.90      0.90      0.90       922
                             Outros       0.86      0.85      0.86       549
       Roubo / Relatório de disputa       0.82      0.90      0.86      1204
         Serviços de conta bancária       0.91      0.89      0.90      1303

                           accuracy                           0.88      5268
                          macro avg       0.88      0.88      0.88      5268
                       weighted avg       0.89      0.88      0.88      5268

